In [ ]:
# import
import logging
import os
import sys

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import nest_asyncio
import openai
from dotenv import load_dotenv
from IPython.display import Markdown, display
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import (
    ServiceContext,
    SimpleDirectoryReader,
    SimpleKeywordTableIndex,
    SummaryIndex,
    VectorStoreIndex,
    set_global_service_context,
)
from llama_index.composability import ComposableGraph
from llama_index.embeddings import HuggingFaceEmbedding, OpenAIEmbedding
from llama_index.llms import OpenAI
from llama_index.node_parser import SentenceSplitter
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import ChromaVectorStore

# ChromaDB client 
chroma_client = chromadb.Client()








# OpenAI API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
openai.api_key = OPENAI_API_KEY

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-ada-002"
            )


hf_ef = embedding_functions.HuggingFaceEmbeddingFunction(
                api_key=HUGGINGFACE_API_KEY,
                model_name="sentence-transformers/all-mpnet-base-v2"
            )

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# embed_model = OpenAIEmbedding(embed_batch_size=10)

hf_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)



service_context = ServiceContext.from_defaults(embed_model=embed_model)

# optionally set a global service context
set_global_service_context(service_context)




nest_asyncio.apply()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
reader = SimpleDirectoryReader("../necrovox_docs/")
documents = reader.load_data()


nodes = SentenceSplitter().get_nodes_from_documents(documents)

In [ ]:
collection = chroma_client.get_or_create_collection(
    name="necromunda_hf",
    embedding_function=hf_ef,
)

In [ ]:
nodes = SentenceSplitter().get_nodes_from_documents(documents)

In [ ]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=hf_embed_model)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

# Query Data
query_engine = index.as_query_engine()
response = query_engine.query("How do you make an injury roll?")
display(Markdown(f"<b>{response}</b>"))

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [ ]:
train_nodes = load_corpus(["..\\necrovox_docs\\rules.md", "..\\necrovox_docs\\gangs.md"], verbose=True)

In [ ]:
train_nodes = load_corpus(["..\\necrovox_docs\\rules.md", "..\\necrovox_docs\\gangs.md"], verbose=True)

In [ ]:
from llama_index.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)

In [ ]:
# setup prompts - specific to StableLM
from llama_index.prompts import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
%pip install accelerate

In [10]:
from llama_index.llms import HuggingFaceLLM
import torch

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Writer/camel-5b-hf",
    model_name="Writer/camel-5b-hf",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)
service_context = ServiceContext.from_defaults(chunk_size=512, llm=llm)

Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/53acde11911aeca10bccfe0a26c001702ad812b0d2f589c4999bf81e6c9a2404?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00003.bin%3B+filename%3D%22pytorch_model-00001-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705881140&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4MTE0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzUzYWNkZTExOTExYWVjYTEwYmNjZmUwYTI2YzAwMTcwMmFkODEyYjBkMmY1ODljNDk5OWJmODFlNmM5YTI0MDQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Wm%7EEZeYmM88MozSH68Dr0y6I6%7EFBIZbMkTmZoEIg07ou1UQW471WMGrYgaMdY5dEh1cuGS2-lYGbh4k8ZM2UNwdWi8H0b4XbFqk4Z4W1FC80yJXW%7EXz0B2Okb4O8A

Trying to resume download...
Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/53acde11911aeca10bccfe0a26c001702ad812b0d2f589c4999bf81e6c9a2404?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00003.bin%3B+filename%3D%22pytorch_model-00001-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705881140&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4MTE0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzUzYWNkZTExOTExYWVjYTEwYmNjZmUwYTI2YzAwMTcwMmFkODEyYjBkMmY1ODljNDk5OWJmODFlNmM5YTI0MDQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Wm%7EEZeYmM88MozSH68Dr0y6I6%7EFBIZbMkTmZoEIg07ou1UQW471WMGrYgaMdY5dEh1cuGS2-lYGbh4k8ZM2UNwdWi8H0b4XbFq

pytorch_model-00001-of-00003.bin:  74%|#######4  | 7.43G/10.0G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/53acde11911aeca10bccfe0a26c001702ad812b0d2f589c4999bf81e6c9a2404?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00003.bin%3B+filename%3D%22pytorch_model-00001-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705881140&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4MTE0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzUzYWNkZTExOTExYWVjYTEwYmNjZmUwYTI2YzAwMTcwMmFkODEyYjBkMmY1ODljNDk5OWJmODFlNmM5YTI0MDQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Wm%7EEZeYmM88MozSH68Dr0y6I6%7EFBIZbMkTmZoEIg07ou1UQW471WMGrYgaMdY5dEh1cuGS2-lYGbh4k8ZM2UNwdWi8H0b4XbFqk4Z4W1FC80yJXW%7EXz0B2Okb4O8A

Trying to resume download...
Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/53acde11911aeca10bccfe0a26c001702ad812b0d2f589c4999bf81e6c9a2404?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00003.bin%3B+filename%3D%22pytorch_model-00001-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705881140&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4MTE0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzUzYWNkZTExOTExYWVjYTEwYmNjZmUwYTI2YzAwMTcwMmFkODEyYjBkMmY1ODljNDk5OWJmODFlNmM5YTI0MDQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Wm%7EEZeYmM88MozSH68Dr0y6I6%7EFBIZbMkTmZoEIg07ou1UQW471WMGrYgaMdY5dEh1cuGS2-lYGbh4k8ZM2UNwdWi8H0b4XbFq

pytorch_model-00001-of-00003.bin:  76%|#######5  | 7.59G/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/4a2dffb78952892cf0654fc7a8d3e5eac8cdf1f9e2b0ff061535ca5afde97264?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00002-of-00003.bin%3B+filename%3D%22pytorch_model-00002-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705884749&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4NDc0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzRhMmRmZmI3ODk1Mjg5MmNmMDY1NGZjN2E4ZDNlNWVhYzhjZGYxZjllMmIwZmYwNjE1MzVjYTVhZmRlOTcyNjQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=cazn60HbF4GF56X5YLeXf5s4mXk3A7DuQvReWSND3PmdNmDiw9PU9VyiIfqn29Yn1zqb-SKEVuBXfXollQBj8uBI2d8aYcjGm4wi86QQoHw1jRSTgILSjY-P%7EI9C2md4k

Trying to resume download...
Error while downloading from https://cdn-lfs.huggingface.co/repos/bc/f0/bcf0c3e0cd7c59f7f6ecc0d5f5f21c4e1249c257f1454bf23d6a5a6433f319ec/4a2dffb78952892cf0654fc7a8d3e5eac8cdf1f9e2b0ff061535ca5afde97264?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00002-of-00003.bin%3B+filename%3D%22pytorch_model-00002-of-00003.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705884749&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTg4NDc0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iYy9mMC9iY2YwYzNlMGNkN2M1OWY3ZjZlY2MwZDVmNWYyMWM0ZTEyNDljMjU3ZjE0NTRiZjIzZDZhNWE2NDMzZjMxOWVjLzRhMmRmZmI3ODk1Mjg5MmNmMDY1NGZjN2E4ZDNlNWVhYzhjZGYxZjllMmIwZmYwNjE1MzVjYTVhZmRlOTcyNjQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=cazn60HbF4GF56X5YLeXf5s4mXk3A7DuQvReWSND3PmdNmDiw9PU9VyiIfqn29Yn1zqb-SKEVuBXfXollQBj8uBI2d8aYcjGm4wi86

pytorch_model-00002-of-00003.bin:  26%|##6       | 2.61G/9.99G [00:00<?, ?B/s]

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)